<a href="https://colab.research.google.com/github/FedericoRaschiatore0123/GATr_Deep_Learning_Project/blob/main/GATr_deep_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/FedericoRaschiatore0123/GATr_Deep_Learning_Project.git

Cloning into 'GATr_Deep_Learning_Project'...
remote: Enumerating objects: 1613, done.
remote: Counting objects: 100% (1613/1613), done.
remote: Compressing objects: 100% (1608/1608), done.
remote: Total 1613 (delta 61), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (1613/1613), 448.76 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Updating files: 100% (2161/2161), done.


In [5]:
#librerie
!pip install h5py
!pip install trimesh
!pip install torch_geometric

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch_geometric.data
from torch_geometric.data import Data
from torch.utils.data import random_split, ConcatDataset, Subset

import h5py
import os
import trimesh

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

single_path ='/content/GATr_Deep_Learning_Project/single/'
bifurcating_path = '/content/GATr_Deep_Learning_Project/bifurcating/'

In [6]:
print(len(os.listdir(single_path)))
print(len(os.listdir(bifurcating_path)))

1100
1058


In [11]:
class Preprocesing():
  def __init__(self, path_single, path_bifurcating):

    self.single_data_path = path_single
    self.bifurcating_data_path = path_bifurcating

  def build_datasets(self):

    single_dataset = self.load_data(self.single_data_path,  bifurcating=False)
    bifurcating_dataset = self.load_data(self.bifurcating_data_path,  bifurcating=True)

    combined_dataset = self.shuffle_and_combine_datasets(single_dataset, bifurcating_dataset)

    return self.split_dataset(combined_dataset)


  def load_data(self, path, bifurcating=False):
    data = []
    for file in os.listdir(path):
      with h5py.File(path + file, 'r') as input_file:
        data.append(

            Data(
              wss=torch.from_numpy(input_file['wss'][()]),
              pos=torch.from_numpy(input_file['pos'][()]),
              pressure=torch.from_numpy(input_file['pressure'][()]),
              face=torch.from_numpy(input_file['face'][()].T).long(),
              inlet_index=torch.from_numpy(input_file['inlet_idcs'][()])
          ))

    label = torch.ones(len(data)) if bifurcating else torch.zeros(len(data))

    return list(zip(data, label))


  def shuffle_and_combine_datasets(self, dataset1, dataset2):

      combined_dataset = ConcatDataset([dataset1, dataset2])
      indices = torch.randperm(len(combined_dataset))
      shuffled_dataset = Subset(combined_dataset, indices)

      return shuffled_dataset

  def split_dataset(self, dataset, train_ratio=0.7, validation_ratio=0.2, test_ratio=0.1):

      total_size = len(dataset)
      train_size = int(total_size * train_ratio)
      validation_size = int(total_size * validation_ratio)
      test_size = total_size - train_size - validation_size

      train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

      return train_dataset, validation_dataset, test_dataset


dataset = Preprocesing(single_path, bifurcating_path)
train_dataset, validation_dataset, test_dataset = dataset.build_datasets()

print(train_dataset)
print(validation_dataset)
print(test_dataset)

In [18]:
import random

single_list = os.listdir(single_path)

with h5py.File(single_path + single_list[random.randint(0, len(single_list))] , 'r') as file:

    vertices = np.array(file['pos'])
    faces = np.array(file['face'])

mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.show()

In [24]:
bifurcating_list = os.listdir(bifurcating_path)

with h5py.File("/content/GATr_Deep_Learning_Project/bifurcating/sample_0010.hdf5", 'r') as file:
    vertices = np.array(file['pos'])
    faces = np.array(file['face'])

mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.show()

In [19]:
with h5py.File('/content/sample_0000.hdf5', 'r') as input_file:
  print(input_file[:][:])

[[    1     0     2]
 [    2     3     4]
 [    4     5     6]
 ...
 [10468 10353 10328]
 [10380 10468 10328]
 [10432 10353 10468]]
